In [1]:
# load gt questions
from collections import defaultdict, Counter

path_qs = '../data/cast19/train_questions.tsv'

t_qs = defaultdict(list)
with open(path_qs, 'r') as f:
    for line in f:
        q_id, q = line.strip('\n').split('\t')
        topic = q_id.split('_')[0]
        t_qs[topic].append(q)
print(len(t_qs), 'topics')

80 topics


In [2]:
# count words within and across topics
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer 
from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer() 
# from nltk.tokenize import word_tokenize 
   
ps = PorterStemmer() 

tokenizer = RegexpTokenizer(r'\w+')
stopwords = stopwords.words('english')

topic_words = {}
for topic, qs in t_qs.items():
    words = Counter()
    for q in qs:
#         question_words = tokenizer.tokenize(q.lower())
#         question_words = [ps.stem(w) for w in tokenizer.tokenize(q.lower())]
        question_words = [lemmatizer.lemmatize(w) for w in tokenizer.tokenize(q.lower()) if w not in stopwords]
        words.update(question_words)
        
    frequent_words = [w for w in words if words[w] > 1 and w not in stopwords]
    topic_words[topic] = frequent_words

all_words = [w for words in topic_words.values() for w in words]

counter = Counter(all_words)
repeated_words = [w for w in counter if counter[w] > 5]
print(repeated_words)

['important', 'tell']


In [3]:
# represent questions as bows

action_sequences = []
for topic, frequent_words in topic_words.items():
    question_sequence = []
    qs = t_qs[topic]
#     print(len(qs))
    for q in qs:
        
        question_words = set([ps.stem(w) for w in tokenizer.tokenize(q.lower()) if w not in stopwords+repeated_words])# if w not in repeated_words and w in frequent_words])
#         
        if question_sequence:
            previous_q = question_sequence[-1]
            overlap = question_words & previous_q
            if not overlap:
                action = 'Drift'
            else:
                added = question_words - previous_q
                removed = previous_q - question_words
                if added:
                    if removed:
                        action = 'Explore'
                    else:
                        action = 'Specialise'
                elif removed:
                    action = 'Generalise'
                else:
                    action = 'Follow'
            print(action)
            action_sequences[-1].append(action)
        else:
            print('Initialise')
            action_sequences.append(['Initialise'])
#         print(q)
#         print(question_words)
        question_sequence.append(question_words)
    print('\n')
#     break

Initialise
Specialise
Specialise
Explore
Explore
Drift
Explore
Explore
Explore
Generalise
Explore
Explore


Initialise
Explore
Explore
Explore
Explore
Explore
Explore
Explore
Explore
Explore
Explore


Initialise
Explore
Drift
Drift
Specialise
Explore
Explore
Explore
Explore
Explore
Explore


Initialise
Specialise
Generalise
Explore
Explore
Explore
Explore
Explore
Explore


Initialise
Explore
Explore
Explore
Explore
Explore
Explore
Specialise
Explore
Specialise


Initialise
Specialise
Explore
Specialise
Explore
Explore
Explore
Explore
Explore


Initialise
Drift
Explore
Explore
Explore
Drift
Specialise
Explore
Explore
Explore
Explore


Initialise
Specialise
Explore
Explore
Explore
Explore
Drift


Initialise
Specialise
Explore
Explore
Explore
Drift
Specialise
Explore
Specialise


Initialise
Specialise
Drift
Explore
Drift
Drift
Drift
Explore
Drift


Initialise
Specialise
Explore
Explore
Explore
Explore
Explore
Explore


Initialise
Explore
Explore
Explore
Explore
Explore
Explore
Explore
Exp